<a href="https://colab.research.google.com/github/PawinData/UC/blob/SFBA/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re
from pickle import dump, load
from functions import str_to_day, day_to_str

This is to create a dictionary of date $\rightarrow$ feature dataframe within a certain time window. Each feature dataframe, mapped from a date, has $M$ rows (features) and $N$ columns (counties).

**FEATURES_raw.p**: date $\rightarrow$ raw feature dataframe

**FEATURES_normal.p**: date $\rightarrow$ normalized feature dataframe

Both are stored in pickle files.

In [0]:
# set up time window of feature dataframes
start = "2020-02-01"
end = "2020-05-02"
Time_Window = [day_to_str(day) for day in range(str_to_day(start), 1+str_to_day(end))]

In [0]:
# columns of a feature dataframe
Counties = ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", 
            "San Mateo", "Santa Clara", "Solano", "Sonoma"]
Counties.sort()

# a dictionary of date --> M-by-N feature dataframe
FEATURES = dict()

# Meteorological Data

In [0]:
# load datasets
TEMP = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/WeatherData/temperatureData2.csv",
                   index_col=0)
TEMP = TEMP[Counties]

WIND = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/WeatherData/windSpeedData2.csv",
                   index_col=0)
WIND = WIND[Counties]

PREP = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/WeatherData/precipitationData2_SUM.csv",
                   index_col=0)
PREP = PREP[Counties]

In [0]:
# construct feature dataframe
for date in Time_Window:
    mtx = np.array([TEMP.loc[date,:].tolist(), 
                    WIND.loc[date,:].tolist(), 
                    PREP.loc[date,:].tolist()])
    FEATURES[date] = pd.DataFrame(mtx, columns=Counties,
                                  index=["temperature", "wind_speed", "precipitation"])

In [0]:
# remove redundant datasets
del TEMP, WIND, PREP

# Demographic Data

In [0]:
# load datasets
UME = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/UnemploymentRate.csv",
                  index_col=0)

INCOME = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/randomData.csv",
                     index_col=0)
INCOME.columns = ["population","density","income_per_capita","median_household_income"]

CRIME = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/CrimeData/crime.csv",
                    index_col=0)

In [0]:
# a function to extract month from a date
def Month(date):
    mm = re.findall('^2020-([0-9]+)-', date)
    mm = int(mm[0])
    if mm==1:
        month = "JAN"
    elif mm==2:
        month = "FEB"
    elif mm==3:
        month = "MAR"
    else:
        month = "APR"
    return month

In [0]:
# add features to the dictionary
for date in Time_Window:
    month = Month(date)
    mtx = np.array([UME[month].tolist(),
                   CRIME["crime_rate_per_100000"].tolist(),
                   INCOME["income_per_capita"].tolist()])
    df = pd.DataFrame(mtx, columns=Counties,
                      index=["unemployment_rate","crime_rate","income_per_capita"])
    FEATURES[date] = FEATURES[date].append(df)

In [0]:
# remove redundant datesets
del UME, INCOME, CRIME

# Foursquare Data (Social Distancing)

In [11]:
# load dataset
F = pd.read_csv("https://raw.githubusercontent.com/PawinData/UC/SFBA/Data/Foursquare/fourSquareData.csv")
F.head()

,date,county,categoryname,visits
0,2020-02-01,San Mateo,Clothing Stores,10414
1,2020-02-01,San Mateo,Medical Centers,10314
2,2020-02-01,San Mateo,Banks,14876
3,2020-02-01,San Mateo,Pharmacies,24837
4,2020-02-01,San Mateo,Salons / Barbershops,11699


In [12]:
# print all categories of POIs
POI = F.categoryname.tolist()
POI.sort()
POI = np.array(POI)
POI = np.unique(POI)
print("There are",len(POI),"categories of POIs.")
print(POI)

There are 206 categories of POIs.
['Airport' 'Airport Gates' 'Airport Lounges' 'Airport Services'
 'Airport Terminals' 'Airport Trams' 'Airports' 'Alcohol'
 'American Restaurants' 'Art Galleries' 'Art Museums'
 'Arts & Entertainment' 'Asian Restaurants' 'Auto Dealerships'
 'Automotive Shops' 'BBQ Joints' 'Bagel Shops' 'Bakeries' 'Banks' 'Bars'
 'Baseball Fields' 'Baseball Stadiums' 'Basketball Stadiums' 'Beach'
 'Beaches' 'Beer Gardens' 'Big Box Stores' 'Boats or Ferries' 'Bookstores'
 'Breakfast Spots' 'Breweries' 'Bridges' 'Bubble Tea Shops' 'Buildings'
 'Burger Joints' 'Bus' 'Bus Lines' 'Bus Stations' 'Bus Stops'
 'Business Centers' 'Cafés' 'Car Washes' 'Casinos' 'Chinese Restaurants'
 'Chocolate Shops' 'Churches' 'Clothing Stores' 'Cocktail Bars'
 'Coffee Shops' 'College Academic Buildings' 'College Cafeterias'
 'College Engineering Buildings' 'College Libraries'
 'Colleges & Universities' 'Community Colleges' 'Convenience Store'
 'Convenience Stores' 'Convention Centers' 'Corporat

## Classify POIs into 4 types

In [0]:
# judge if a string contains any substring in a list
def kw_classifier(st, lst):
    judge = False
    for element in lst:
        if element in st:
            judge = True
            break
    return judge

In [14]:
# Rough classifier by keywords
Commercial = list()
Edu_Med = list()
Transport = list()
Recreational = list()

keywords = {"Edu_Med":["school","college","universit","librar","med","doctor",
                       "dentist","hospital","government","student"],
            "Commercial":["store","mall","shop","restaurant","grocery","bar","plaza",
                          "café","deli","dealer","night","bbq","baker","beer","breakfast",
                          "dine","market","food","pharma","place","pub","lounge","joint",
                          "office","theater"],
            "Transport":["airport","station","bus","boat","pier","hotel","marina","travel"],
            "Recreational":["art","beach","field","stadium","gym","zoo","dog","fitness",
                            "entertainment","arena","outdoor","play","golf", "landmark"]}

poi = POI.tolist()
for element in POI.tolist():
    ele = element.lower().strip()
    if kw_classifier(ele, keywords["Edu_Med"]):
        Edu_Med.append(element)
        poi.remove(element)
    elif kw_classifier(ele, keywords["Transport"]):
        Transport.append(element)
        poi.remove(element)
    elif kw_classifier(ele, keywords["Commercial"]):
        Commercial.append(element)
        poi.remove(element)
    elif kw_classifier(ele, keywords["Recreational"]):
        Recreational.append(element)
        poi.remove(element)

print("Edu&Med:",len(Edu_Med))
print("Commercial:",len(Commercial))
print("Transport:",len(Transport))
print("Recreational:",len(Recreational))
print("There are",len(poi),"keywords left:")
print(np.array(poi))

Edu&Med: 23
Commercial: 95
Transport: 24
Recreational: 23
There are 41 keywords left:
['Alcohol' 'Banks' 'Breweries' 'Bridges' 'Buildings' 'Car Washes'
 'Casinos' 'Churches' 'Convention Centers' 'Corporate Cafeterias'
 'Coworking Spaces' 'Event Spaces' 'Factories' 'Farms' 'Gardens'
 'Historic Sites' 'Housing Developments' 'Intersections' 'Lakes'
 'Laundry Services' 'Meeting Rooms' 'Military Bases' 'Music Venues'
 'Nail Salons' 'National Parks' 'Nature Preserves' 'Parking' 'Parks'
 'Pools' 'Rental Car Locations' 'Residences' 'Scenic Lookouts' 'Spas'
 'Spiritual Center' 'Sports' 'State / Provincial Parks'
 'Storage Facilities' 'Trails' 'Vineyards' 'Wineries' 'Yoga Studios']


In [15]:
# Prune classifier manually
Commercial.extend(["Alcohol","Banks","Car Washes","Casinos","Corporate Cafeterias",
                  "Coworking Spaces", "Event Spaces", "Laundry Services",
                  "Music Venues", "Nail Salons", "Retal Car Locations", "Spas",
                  "Meeting Rooms"])
Recreational.extend(["Gardens","Historic Sites","Lakes","Pools","Sports",
                    "Yoga Studios"])
Transport.append("Intersections")

print("Edu_Med:", Edu_Med)
print("")
print("Commercial:", Commercial)
print("")
print("Transport:", Transport)
print("")
print("Recreational:", Recreational)

Edu_Med: ['College Academic Buildings', 'College Cafeterias', 'College Engineering Buildings', 'College Libraries', 'Colleges & Universities', 'Community Colleges', "Dentist's Offices", "Doctor's Offices", 'Elementary Schools', 'Government', 'Government Buildings', 'High Schools', 'Hospitals', 'Libraries', 'Medical', 'Medical Centers', 'Medical Schools', 'Mediterranean Restaurants', 'Middle Schools', 'School', 'Schools', 'Student Centers', 'Universities']

Commercial: ['American Restaurants', 'Asian Restaurants', 'Auto Dealerships', 'Automotive Shops', 'BBQ Joints', 'Bagel Shops', 'Bakeries', 'Bars', 'Beer Gardens', 'Big Box Stores', 'Bookstores', 'Breakfast Spots', 'Bubble Tea Shops', 'Burger Joints', 'Cafés', 'Chinese Restaurants', 'Chocolate Shops', 'Clothing Stores', 'Cocktail Bars', 'Coffee Shops', 'Convenience Store', 'Convenience Stores', 'Cosmetics Shops', 'Delis / Bodegas', 'Department Stores', 'Dessert Shops', 'Dim Sum Restaurants', 'Diners', 'Discount Stores', 'Dive Bars', '

## Create feature dataframes

In [0]:
edu_med = 0
commercial = 0
transport = 0
recreational = 0
d = dict()

for date in Time_Window:
    DATA = F.loc[F["date"]==date]
    for cnty in Counties:  # count the number of visits for each type of POI in each county
        data = DATA.loc[DATA["county"]==cnty]
        B = data[["categoryname","visits"]].to_numpy()
        for POI,num in B:
            if POI in Edu_Med: edu_med += num 
            if POI in Commercial: commercial += num 
            if POI in Transport: transport += num
            if POI in Recreational: recreational += num 
        d[cnty] = [edu_med, commercial, transport, recreational]
    d = pd.DataFrame(d, index=["Edu&Med","Commercial","Transport","Recreational"])
    FEATURES[date] = FEATURES[date].append(d)

In [0]:
del F   # remove redundant objects

# Normalization

In [34]:
# store the feature dataframes
dump(FEATURES, open("FEATURES_raw.p","wb"))

FEATURES[start]     # print out the feature dataframe for the first day

,Alameda,Contra Costa,Marin,Napa,San Francisco,San Mateo,Santa Clara,Solano,Sonoma
temperature,2.826250e+02,2.826250e+02,2.859250e+02,2.826250e+02,2.859250e+02,2.859500e+02,2.859500e+02,2.826250e+02,2.859250e+02
wind_speed,-1.037500e+00,-5.016667e-01,1.419583e+00,1.475000e-01,3.445833e-01,-3.041667e-01,-1.443333e+00,-1.135000e+00,5.708333e-01
precipitation,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
unemployment_rate,3.000000e+00,3.100000e+00,2.300000e+00,3.200000e+00,2.300000e+00,2.100000e+00,2.600000e+00,3.900000e+00,2.800000e+00
crime_rate,7.536936e+02,4.006562e+02,1.896542e+02,3.456238e+02,2.654077e+02,7.014217e+02,2.353577e+02,4.345697e+02,3.620019e+02
income_per_capita,3.643900e+04,3.877000e+04,5.800400e+04,3.509200e+04,4.998600e+04,4.719800e+04,4.266600e+04,2.913200e+04,3.336100e+04
Edu&Med,3.203650e+05,4.878590e+05,5.055160e+05,5.055160e+05,6.903490e+05,8.451740e+05,1.403718e+06,1.438226e+06,1.475576e+06
Commercial,3.313228e+06,5.194267e+06,5.562107e+06,5.787948e+06,8.781956e+06,1.082928e+07,1.536301e+07,1.608541e+07,1.696995e+07
Transport,5.010300e+05,6.737580e+05,7.052300e+05,7.638980e+05,1.331759e+06,2.354021e+06,2.934537e+06,3.029060e+06,3.110246e+06
Recreational,5.397720e+05,8.624970e+05,9.812710e+05,1.014224e+06,1.871061e+06,2.225947e+06,2.942989e+06,3.038579e+06,3.165341e+06


In [0]:
FEATURES = load(open("FEATURES_raw.p","rb"))

for date in Time_Window:
    features = FEATURES[date].to_numpy()
    IND = FEATURES[date].index
    rows,cols = features.shape
    for i in range(rows):   # normalize each row of a feature dataframe
        Row = features[i,:]
        if not np.std(Row)==0:
            Row = (Row - np.mean(Row)) / np.std(Row)
        features[i,:] = Row
    FEATURES[date] = pd.DataFrame(features, columns=Counties, index=IND)

In [43]:
FEATURES[start]  # print out the normalized feature dataframe for the first day

,Alameda,Contra Costa,Marin,Napa,San Francisco,San Mateo,Santa Clara,Solano,Sonoma
temperature,-1.118017,-1.118017,0.888334,-1.118017,0.888334,0.903533,0.903533,-1.118017,0.888334
wind_speed,-0.941856,-0.327920,1.873368,0.415868,0.641678,-0.101633,-1.406843,-1.053567,0.900906
precipitation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
unemployment_rate,0.356339,0.544988,-0.964210,0.733638,-0.964210,-1.341510,-0.398261,2.054187,-0.020961
crime_rate,1.852394,-0.049368,-1.186006,-0.345820,-0.777932,1.570813,-0.939807,0.133319,-0.257593
income_per_capita,-0.549574,-0.279543,1.948588,-0.705615,1.019756,0.696785,0.171783,-1.396041,-0.906139
Edu&Med,-1.217758,-0.834442,-0.794034,-0.794034,-0.371037,-0.016715,1.261533,1.340505,1.425982
Commercial,-1.301163,-0.921818,-0.847636,-0.802092,-0.198296,0.214583,1.128890,1.274575,1.452958
Transport,-1.137155,-0.974889,-0.945323,-0.890209,-0.356744,0.603599,1.148953,1.237750,1.314019
Recreational,-1.337517,-1.007838,-0.886505,-0.852842,0.022459,0.384992,1.117486,1.215136,1.344629


In [0]:
# store the dictionary of normalized feature dataframes
dump(FEATURES, open("FEATURES_normal.p","wb"))